Nama : Stevani Dwi Utomo

NIM : 24/546969/PPA/06865

In [9]:
# install library newsapi dan NLTK
!pip install newsapi-python
!pip install nltk

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dot, Flatten
import pandas as pd
from newsapi import NewsApiClient
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
def news_scrape(YOUR_API_KEY, q, from_param, to, language='en', sort_by='popularity', max_page=1):
  # Init NewsAPI
  newsapi = NewsApiClient(api_key=YOUR_API_KEY)
  df_all = pd.DataFrame()
  for i in range(1,max_page+1):
    all_articles = newsapi.get_everything(q=q, from_param=from_param, to=to,
                                          language=language, sort_by=sort_by, page=i)
    df_tmp = pd.json_normalize(all_articles['articles'])
    df_all = pd.concat([df_all, df_tmp], axis=0).reset_index(drop=True)

  df_all = df_all.drop_duplicates()
  return df_all

# cleansing text
def text_cleansing(text):
    # Lowercasing
    text = text.lower()
    # Remove the pattern "[+digits chars]"
    text = re.sub(r'\[\+\d+\schars\]', '', text).strip()
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    # Join back to a string
    cleaned_text = ' '.join(lemmas)
    return cleaned_text

In [24]:
YOUR_API_KEY = "cf700d4d4a474353b0cdb01a18e8f70a"

df_all = news_scrape(YOUR_API_KEY, q="technology", from_param="2025-02-14", to="2025-03-14",
                     language="en", sort_by="popularity", max_page=5)

# Pastikan ada data
if df_all.empty:
    raise ValueError("Tidak ada berita yang diambil dari NewsAPI!")

# Terapkan pembersihan teks
df_all_cont = df_all["content"].astype(str).apply(text_cleansing)

# Gabungkan semua teks menjadi satu dokumen besar
corpus = " ".join(df_all_cont)

# Tokenisasi teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # +1 untuk padding

# Konversi teks ke dalam format angka
sequences = tokenizer.texts_to_sequences([corpus])[0]

# Validasi: Pastikan ada cukup kata
if len(sequences) < 2:
    raise ValueError("Data tidak cukup untuk Skipgram. Tambahkan lebih banyak berita!")

# Skipgram pairs
pairs, labels = skipgrams(sequences, vocabulary_size=vocab_size, window_size=2)

# Periksa apakah pairs tidak kosong sebelum unpacking
if len(pairs) == 0:
    raise ValueError("Skipgram tidak menghasilkan pasangan kata. Periksa preprocessing data!")

target_words, context_words = zip(*pairs)

# Bangun model Skipgram dengan dua input
def build_model(vocab_size, embedding_dim):
    target_input = Input(shape=(1,))
    context_input = Input(shape=(1,))

    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1)

    target_embedding = embedding(target_input)
    context_embedding = embedding(context_input)

    dot_product = Dot(axes=2)([target_embedding, context_embedding])
    dot_product = Flatten()(dot_product)

    model = Model(inputs=[target_input, context_input], outputs=dot_product)
    model.compile(optimizer="adam", loss="binary_crossentropy")
    return model

# Gunakan variasi dimensi embedding
embedding_dims = [50, 100, 200]

for embedding_dim in embedding_dims:
    print(f"Training model dengan embedding_dim = {embedding_dim}")

    model = build_model(vocab_size, embedding_dim)

    target_words = np.array(target_words)
    context_words = np.array(context_words)
    labels = np.array(labels)

    model.fit([target_words, context_words], labels, epochs=5, batch_size=4, verbose=1)

    # Ambil embedding untuk analisis
    embedding_layer = model.layers[2]  # Layer embedding ada di index ke-2
    word_embeddings = embedding_layer.get_weights()[0]

    # Tampilkan embedding untuk kata 'technology' jika ada dalam vocab
    if "technology" in word_index:
        print(f"Embedding untuk 'technology': {word_embeddings[word_index['bitcoin']]}")
    else:nb
        print("'technology' tidak ditemukan dalam vocab.")

print("Training selesai!")

Training model dengan embedding_dim = 50
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


30375/30375 ━━━━━━━━━━━━━━━━━━━━ 105s 3ms/step - loss: 3.8821
Epoch 2/5
30375/30375 ━━━━━━━━━━━━━━━━━━━━ 101s 3ms/step - loss: 1.7944
Epoch 3/5
30375/30375 ━━━━━━━━━━━━━━━━━━━━ 147s 3ms/step - loss: 1.3861
Epoch 4/5
30375/30375 ━━━━━━━━━━━━━━━━━━━━ 138s 3ms/step - loss: 1.2197
Epoch 5/5
30375/30375 ━━━━━━━━━━━━━━━━━━━━ 139s 3ms/step - loss: 1.1566
Embedding untuk 'technology': [ 0.26697364 -0.08389466  0.18688537  0.21166794  0.13945119  0.31528673
  0.15733288 -0.0796235  -0.2004255   0.2130853   0.00213994 -0.30655682
  0.23404074  0.02579229  0.09981859  0.13985477  0.04212297 -0.3528547
 -0.00685172 -0.28958303  0.00352312  0.08599048  0.0708724  -0.18475132
  0.05354453  0.08616711 -0.08317644  0.06745519 -0.3158485   0.40982673
  0.07193493 -0.05263796 -0.3242715   0.30542025 -0.30222037 -0.07026853
  0.13202691  0.42085418 -0.14766654  0.04025715  0.07398832  0.00381349
 -0.09755365 -0.01969904  0.1675041  -0.20397899 -0.32177308  0.19935483
  0.14475025 -0.25608513]
Training mo

In [27]:
# Ambil nama layer embedding secara otomatis
embedding_layer_name = [layer.name for layer in model.layers if 'embedding' in layer.name][0]

# Ambil bobot embedding dari model
embeddings = model.get_layer(embedding_layer_name).get_weights()[0]

In [28]:
# Fungsi untuk mendapatkan kata yang mirip berdasarkan vektor embedding
def get_similar_words(target_word, top_n=5):
    if target_word not in word2idx:
        print(f"'{target_word}' tidak ditemukan dalam vocabulary.")
        return None

    word_idx = word2idx[target_word]

    # Ambil nama layer embedding yang sesuai
    embedding_layer_name = [layer.name for layer in model.layers if 'embedding' in layer.name][0]

    # Ambil bobot embedding dari model
    embeddings = model.get_layer(embedding_layer_name).get_weights()[0]

    # Pastikan indeks kata valid
    if word_idx >= len(embeddings):
        print(f"Indeks '{target_word}' di luar jangkauan embedding.")
        return None

    word_vec = embeddings[word_idx]

    # Hitung kemiripan kosinus (dot product)
    similarities = np.dot(embeddings, word_vec)

    # Urutkan indeks berdasarkan kemiripan (dari yang paling mirip)
    closest_indices = np.argsort(similarities)[::-1][1:top_n+1]

    return [idx2word[idx] for idx in closest_indices if idx in idx2word]

# Coba cari kata-kata yang mirip dengan 'technology'
print(get_similar_words("technology"))


['by', 'first', 'video', 'president', 'tool']
